In [1]:
%cd ..

/Users/yuqingwang/Desktop/Time Series Final


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
import statsmodels.api as sm
from scipy.optimize import minimize

In [3]:
def load_data(file):
    Stock = pd.read_csv(file)
    Stock['Date'] = pd.to_datetime(Stock['Date'])
    Stock.set_index(['Date'],inplace=True)
    Stock = Stock['Close']
    Stock_return = np.log(Stock/Stock.shift())
    Stock_return.dropna(inplace = True)
    return Stock_return

In [4]:
NFLX_return = load_data('data/NFLX.csv')

In [5]:
NFLX_return_mean = NFLX_return.loc[NFLX_return.index<pd.to_datetime('2019-10-30')].mean()

## Constant Volatiltiy

In [6]:
vol = np.std(NFLX_return.loc[NFLX_return.index<pd.to_datetime('2019-10-30')])
vol

0.024824316605013056

In [7]:
vol*vol

0.0006162466949059269

## ARCH(1)

In [8]:
parameter_ARCH = np.array([vol,0.3,NFLX_return_mean])#alfa_0,alfa_1,miu


def L_ARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in NFLX_return.loc[NFLX_return.index<pd.to_datetime('2019-10-30')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[2])*(i-parameter_0[2])/(sigma2)/2
        sigma2 = parameter_0[0]+parameter_0[1]*(i-parameter_0[2])*(i-parameter_0[2])
    return result

res_ARCH = minimize(L_ARCH, parameter_ARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_ARCH

/Users/yuqingwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


Optimization terminated successfully.
         Current function value: -2192.397476
         Iterations: 230
         Function evaluations: 432


 final_simplex: (array([[0.00057936, 0.06576088, 0.00105597],
       [0.00057936, 0.06576088, 0.00105597],
       [0.00057936, 0.06576088, 0.00105597],
       [0.00057936, 0.06576088, 0.00105597]]), array([-2192.39747573, -2192.39747573, -2192.39747573, -2192.39747573]))
           fun: -2192.3974757294095
       message: 'Optimization terminated successfully.'
          nfev: 432
           nit: 230
        status: 0
       success: True
             x: array([0.00057936, 0.06576088, 0.00105597])

In [9]:
ARCH_estimated_vol = []
sigma2 = vol*vol
for i in NFLX_return:
    sigma2 = res_ARCH.x[0]+res_ARCH.x[1]*(i-res_ARCH.x[2])*(i-res_ARCH.x[2])
    ARCH_estimated_vol.append(sigma2)
ARCH_estimated_vol.pop
ARCH_estimated_vol = pd.DataFrame(ARCH_estimated_vol, index = NFLX_return.index, columns = ['ARCH_estimated_vol'])

In [20]:
#np.sqrt(ARCH_estimated_vol*252).to_pickle('Estimated Volatility/MFLX_ARCH_estimated_vol.pkl')

## GARCH(1,1)

In [11]:
parameter_GARCH = np.array([vol,0.34,0.66,NFLX_return_mean])#alfa_0,alfa_1,beta_1,miu


def L_GARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in NFLX_return.loc[NFLX_return.index<pd.to_datetime('2019-10-30')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[3])*(i-parameter_0[3])/(sigma2)/2
        sigma2 = parameter_0[0]+parameter_0[1]*(i-parameter_0[3])*(i-parameter_0[3])+parameter_0[2]*sigma2
    return result

res_GARCH = minimize(L_GARCH, parameter_GARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_GARCH

/Users/yuqingwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


Optimization terminated successfully.
         Current function value: -2203.326087
         Iterations: 537
         Function evaluations: 920


 final_simplex: (array([[3.62260648e-05, 3.56636769e-02, 9.04695530e-01, 1.07064313e-03],
       [3.62260650e-05, 3.56636768e-02, 9.04695529e-01, 1.07064313e-03],
       [3.62260646e-05, 3.56636766e-02, 9.04695530e-01, 1.07064314e-03],
       [3.62260645e-05, 3.56636768e-02, 9.04695530e-01, 1.07064312e-03],
       [3.62260647e-05, 3.56636768e-02, 9.04695530e-01, 1.07064313e-03]]), array([-2203.32608693, -2203.32608693, -2203.32608693, -2203.32608693,
       -2203.32608693]))
           fun: -2203.3260869344076
       message: 'Optimization terminated successfully.'
          nfev: 920
           nit: 537
        status: 0
       success: True
             x: array([3.62260648e-05, 3.56636769e-02, 9.04695530e-01, 1.07064313e-03])

In [12]:
GARCH_estimated_vol = []
sigma2 = vol*vol
for i in NFLX_return:
    sigma2 = res_GARCH.x[0]+res_GARCH.x[1]*(i-res_GARCH.x[3])*(i-res_GARCH.x[3])+res_GARCH.x[2]*sigma2
    GARCH_estimated_vol.append(sigma2)
GARCH_estimated_vol.pop
GARCH_estimated_vol = pd.DataFrame(GARCH_estimated_vol, index = NFLX_return.index, columns = ['GARCH_estimated_vol'])
GARCH_estimated_vol.mean()

GARCH_estimated_vol    0.000607
dtype: float64

In [21]:
#np.sqrt(GARCH_estimated_vol*252).to_pickle('Estimated Volatility/MFLX_GARCH_estimated_vol.pkl')

## TGARCH

In [14]:
parameter_TGARCH = np.array([0,0.35,0.5,0,0])#alfa_0,alfa_1,beta_!,miu,gamma1

def L_TGARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in NFLX_return.loc[NFLX_return.index<pd.to_datetime('2019-10-30')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[3])*(i-parameter_0[3])/(sigma2)/2
        if i-parameter_0[3]<0:
            S = 1
        else:
            S = 0
        sigma2 = parameter_0[0]+parameter_0[1]*(i-parameter_0[3])*(i-parameter_0[3])+parameter_0[4]*S*(i-parameter_0[3])*(i-parameter_0[3])+parameter_0[2]*sigma2
    return result

res_TGARCH = minimize(L_TGARCH, parameter_TGARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_TGARCH

/Users/yuqingwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log
  import sys


Optimization terminated successfully.
         Current function value: -2210.214513
         Iterations: 661
         Function evaluations: 1095


 final_simplex: (array([[ 7.88975317e-06, -7.44237891e-03,  9.77754452e-01,
         1.22181574e-03,  3.21225585e-02],
       [ 7.88975291e-06, -7.44237833e-03,  9.77754452e-01,
         1.22181570e-03,  3.21225580e-02],
       [ 7.88975314e-06, -7.44237893e-03,  9.77754452e-01,
         1.22181570e-03,  3.21225586e-02],
       [ 7.88975319e-06, -7.44237874e-03,  9.77754452e-01,
         1.22181569e-03,  3.21225582e-02],
       [ 7.88975290e-06, -7.44237885e-03,  9.77754452e-01,
         1.22181583e-03,  3.21225588e-02],
       [ 7.88975310e-06, -7.44237910e-03,  9.77754452e-01,
         1.22181568e-03,  3.21225582e-02]]), array([-2210.21451324, -2210.21451324, -2210.21451324, -2210.21451324,
       -2210.21451324, -2210.21451324]))
           fun: -2210.214513235055
       message: 'Optimization terminated successfully.'
          nfev: 1095
           nit: 661
        status: 0
       success: True
             x: array([ 7.88975317e-06, -7.44237891e-03,  9.77754452e-01,  1.22181574e

In [15]:
TGARCH_estimated_vol = []
sigma2 = vol*vol
for i in NFLX_return:
    if i-res_TGARCH.x[3]<0:
        S = 1
    else:
        S = 0
    sigma2 = res_TGARCH.x[0]+res_TGARCH.x[1]*(i-res_TGARCH.x[3])*(i-res_TGARCH.x[3])+res_TGARCH.x[4]*S*(i-res_TGARCH.x[3])*(i-res_TGARCH.x[3])+res_TGARCH.x[2]*sigma2
    TGARCH_estimated_vol.append(sigma2)
TGARCH_estimated_vol.pop
TGARCH_estimated_vol = pd.DataFrame(TGARCH_estimated_vol, index = NFLX_return.index, columns = ['TGARCH_estimated_vol'])
TGARCH_estimated_vol.mean()

TGARCH_estimated_vol    0.000602
dtype: float64

In [22]:
#np.sqrt(TGARCH_estimated_vol*252).to_pickle('Estimated Volatility/MFLX_TGARCH_estimated_vol.pkl')

## IGARCH

In [17]:
parameter_IGARCH = np.array([vol,0.66,NFLX_return_mean])#alfa_0,beta_1,miu


def L_IGARCH (parameter_0):
    result = 0
    sigma2 = vol*vol
    for i in NFLX_return.loc[NFLX_return.index<pd.to_datetime('2019-10-30')]:
        result = result + np.log(2*math.pi)/2 + np.log(sigma2)/2 + (i-parameter_0[2])*(i-parameter_0[2])/(sigma2)/2
        sigma2 = parameter_0[0]+(1-parameter_0[1])*(i-parameter_0[2])*(i-parameter_0[2])+parameter_0[1]*sigma2
    return result

res_IGARCH = minimize(L_IGARCH, parameter_IGARCH, method='nelder-mead',
                options={'xtol': 1e-9, 'disp': True, 'maxiter': 1000})
res_IGARCH

/Users/yuqingwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


Optimization terminated successfully.
         Current function value: -2192.386121
         Iterations: 629
         Function evaluations: 1145


 final_simplex: (array([[9.22176854e-07, 9.83119634e-01, 1.23599287e-03],
       [9.22176895e-07, 9.83119633e-01, 1.23599275e-03],
       [9.22176900e-07, 9.83119634e-01, 1.23599278e-03],
       [9.22176922e-07, 9.83119633e-01, 1.23599280e-03]]), array([-2192.38612052, -2192.38612052, -2192.38612052, -2192.38612052]))
           fun: -2192.386120522846
       message: 'Optimization terminated successfully.'
          nfev: 1145
           nit: 629
        status: 0
       success: True
             x: array([9.22176854e-07, 9.83119634e-01, 1.23599287e-03])

In [18]:
IGARCH_estimated_vol = []
sigma2 = vol*vol
for i in NFLX_return:
    sigma2 = res_IGARCH.x[0]+(1-res_IGARCH.x[1])*(i-res_IGARCH.x[2])*(i-res_IGARCH.x[2])+res_IGARCH.x[1]*sigma2
    IGARCH_estimated_vol.append(sigma2)
IGARCH_estimated_vol.pop
IGARCH_estimated_vol = pd.DataFrame(IGARCH_estimated_vol, index = NFLX_return.index, columns = ['IGARCH_estimated_vol'])
IGARCH_estimated_vol.mean()

IGARCH_estimated_vol    0.000667
dtype: float64

In [23]:
#np.sqrt(IGARCH_estimated_vol*252).to_pickle('Estimated Volatility/MFLX_IGARCH_estimated_vol.pkl')